## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [7]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.6001,-73.9662,10.72,80,42,5.19,scattered clouds
1,1,Eureka,US,40.8021,-124.1637,38.59,58,0,5.48,clear sky
2,2,Port Alfred,ZA,-33.5906,26.8910,77.76,82,45,14.03,scattered clouds
3,3,Rikitea,PF,-23.1203,-134.9692,77.70,73,46,17.78,scattered clouds
4,4,Torbay,CA,47.6666,-52.7314,24.58,82,75,16.11,broken clouds
5,5,Buala,SB,-8.1450,159.5921,78.40,87,100,7.87,light rain
6,6,Bac Lieu,VN,9.2850,105.7244,77.32,79,88,8.90,overcast clouds
7,7,Xining,CN,36.6167,101.7667,16.02,87,100,3.24,overcast clouds
8,8,Qaanaaq,GL,77.4840,-69.3632,-4.40,68,100,5.79,overcast clouds
9,9,Jamestown,US,42.0970,-79.2353,12.52,65,0,1.01,clear sky


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [32]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Port Alfred,ZA,-33.5906,26.8910,77.76,82,45,14.03,scattered clouds
3,3,Rikitea,PF,-23.1203,-134.9692,77.70,73,46,17.78,scattered clouds
5,5,Buala,SB,-8.1450,159.5921,78.40,87,100,7.87,light rain
6,6,Bac Lieu,VN,9.2850,105.7244,77.32,79,88,8.90,overcast clouds
13,13,Morrinhos,BR,-17.7311,-49.0994,78.44,65,99,5.91,overcast clouds
27,27,Bredasdorp,ZA,-34.5322,20.0403,77.29,60,86,11.50,overcast clouds
31,31,Bengkulu,ID,-3.8004,102.2655,78.66,83,100,3.09,overcast clouds
32,32,Avarua,CK,-21.2078,-159.7750,78.85,94,20,12.66,few clouds
36,36,Presidencia Roque Saenz Pena,AR,-26.7852,-60.4388,76.71,80,100,9.48,overcast clouds
37,37,Namatanai,PG,-3.6667,152.4333,79.41,80,99,2.86,light rain


In [33]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                190
City                   190
Country                189
Lat                    190
Lng                    190
Max Temp               190
Humidity               190
Cloudiness             190
Wind Speed             190
Current Description    190
dtype: int64

In [34]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                189
City                   189
Country                189
Lat                    189
Lng                    189
Max Temp               189
Humidity               189
Cloudiness             189
Wind Speed             189
Current Description    189
dtype: int64

In [35]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Port Alfred,ZA,77.76,scattered clouds,-33.5906,26.8910,
3,Rikitea,PF,77.70,scattered clouds,-23.1203,-134.9692,
5,Buala,SB,78.40,light rain,-8.1450,159.5921,
6,Bac Lieu,VN,77.32,overcast clouds,9.2850,105.7244,
13,Morrinhos,BR,78.44,overcast clouds,-17.7311,-49.0994,
27,Bredasdorp,ZA,77.29,overcast clouds,-34.5322,20.0403,
31,Bengkulu,ID,78.66,overcast clouds,-3.8004,102.2655,
32,Avarua,CK,78.85,few clouds,-21.2078,-159.7750,
36,Presidencia Roque Saenz Pena,AR,76.71,overcast clouds,-26.7852,-60.4388,
37,Namatanai,PG,79.41,light rain,-3.6667,152.4333,


In [36]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
   # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")  

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [38]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df['Hotel Name'] != '']
clean_hotel_df.count()

City                   168
Country                168
Max Temp               168
Current Description    168
Lat                    168
Lng                    168
Hotel Name             168
dtype: int64

In [39]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Port Alfred,ZA,77.76,scattered clouds,-33.5906,26.8910,The Halyards Hotel
3,Rikitea,PF,77.70,scattered clouds,-23.1203,-134.9692,People ThankYou
5,Buala,SB,78.40,light rain,-8.1450,159.5921,Maringe Lagoon Lodge
6,Bac Lieu,VN,77.32,overcast clouds,9.2850,105.7244,Khách sạn Sài Gòn Bạc Liêu
13,Morrinhos,BR,78.44,overcast clouds,-17.7311,-49.0994,Hotel Águia
27,Bredasdorp,ZA,77.29,overcast clouds,-34.5322,20.0403,Bredasdorp Country Manor
31,Bengkulu,ID,78.66,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
32,Avarua,CK,78.85,few clouds,-21.2078,-159.7750,Paradise Inn
36,Presidencia Roque Saenz Pena,AR,76.71,overcast clouds,-26.7852,-60.4388,Hotel Gualok
37,Namatanai,PG,79.41,light rain,-3.6667,152.4333,Seagulls Inn Namatani


In [56]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [54]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and </dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [55]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))